In [1]:
import numpy as np 
import pandas as pd 

import os
import csv
import json
import re
from bs4 import BeautifulSoup
import requests
import random
from datetime import date, timedelta

In [2]:
class Therapies:
    def __init__(self, id, therapy_name, therapy_type):
        self.id = id 
        self.name = therapy_name
        self.type = therapy_type


class Conditions:
    def __init__(self, id, condition_name, condition_type):
        self.id = id 
        self.name = condition_name
        self.type = condition_type
        
class Patients:
    def __init__(self, id, patient_name, patient_gender, patient_age, patient_condition, trials):
        self.id = id
        self.name = patient_name
        self.gender = patient_gender
        self.age = patient_age
        self.conditions = patient_condition
        self.trials = trials
    
class PatientConditions:
    def __init__(self, id, diagnosed, isCured, kind):
        self.id = id
        self.diagnosed = diagnosed
        self.isCured = isCured
        self.kind = kind       
        
        
class PatientTrials:
    def __init__(self, id, start, end, condition, therapy, successful):
        self.id = id
        self.start = start
        self.end = end
        self.condition = condition
        self.therapy = therapy
        self.successful = successful

In [3]:
patientCSV = '../input/p-archive/patient_archive.csv'
conditionJSON = '../input/p-archive/conditions_archive.json'
patientJSON = 'patients.json'

In [4]:
dataSet = {}

In [5]:
def condition():
    dataSet['Conditions'] = []
    condition_data = json.load(open(conditionJSON))
    all_condition = pd.DataFrame(condition_data["sections"])
    condition_types = ['Blood and lymph', 'Brain, nerves and spinal cord', 'Cancer', 'Chromosomal conditions', 'Diabetes', 'Down\'s syndrome', 'Ears, nose and throat', 'Eyes', 'Glands', 'Heart and blood vessels', 'Immune system', 'Infections and poisoning', 'Injuries', 'Kidneys, bladder and prostate', 'Lungs and airways', 'Mental health', 'Mouth', 'Muscle, bone and joints', 'Nutritional', 'Pregnancy and childbirth', 'Sexual and reproductive', 'Skin, hair and nails', 'Stomach, liver and gastrointestinal tract']
        
    i = 1
    for index, condition in all_condition.iterrows():
        condition_name = condition['name']
        condition_type = random.choice(condition_types)
        condition_obj = Conditions('Cond' + str(i), condition_name, condition_type)
        if condition_name not in '':
            dataSet['Conditions'].append(condition_obj.__dict__)
            i = i + 1

In [6]:
def therapy():
    dataSet['Therapies'] = []
    html_texts = requests.get('https://en.wikipedia.org/wiki/List_of_therapies').text
    soup = BeautifulSoup(html_texts, 'lxml')

    therapy_data = soup.find('div', class_ = 'div-col')
    therapies = therapy_data.find_all('li')
    
    therapy_types = ['Abortive Therapy', 'Bridge Therapy', 'Consolidation Therapy', 'Curative Therapy', 'Definitive Therapy', 'Destination Therapy', 'Empiric Therapy', 'Gold Standard Therapy', 'Investigational Therapy', 'Maintenance Therapy', 'Palliative Therapy', 'Preventive Therapy (Prophylactic Therapy)', 'Salvage Therapy (Rescue Therapy)', 'Stepdown Therapy', 'Supportive Therapy', 'Systemic Therapy']
    
    i = 1
    for therapy in therapies:
        therapy_name = therapy.get_text()
        therapy_type = random.choice(therapy_types)
        therapy_obj = Therapies('Th' + str(i), therapy_name, therapy_type)
        if therapy_name not in '':
            dataSet['Therapies'].append(therapy_obj.__dict__)
            i = i + 1

In [7]:
def random_dates(date_range_01, date_range_02):
    dates_diff = date_range_02 - date_range_01
    total_days = dates_diff.days

    res_dates = []
    random_day = random.randrange(total_days)
    start_day = date_range_01 + timedelta(days=random_day)
    res_dates.append(start_day)
    
    
    random_day = random.randint(2, total_days)
    end_day = start_day + timedelta(days=random_day)
    res_dates.append(end_day)

    return res_dates

In [8]:
def patient():
    dataSet['Patients'] = []
    with open(patientCSV) as csvFile:
        csvData = csv.DictReader(csvFile)
    
        i = 1
        for rows in csvData:
            patient_name = rows['FirstName'] + ' ' + rows['LastName']
            patient_gender = rows['Gender']
            patient_age = random.randint(12,100)
            patient_country = rows['Country']
            if patient_name not in '':
                condition_count = random.randint(1, 4)
                patient_condition = []
                
                j = 1
                for x in range(condition_count):
                    trial_count = random.randint(0, 6)
                    diagnosed_date, cured_date = random_dates(date(2000, 1, 1), date(2020, 12, 31))
                    k = 1
                    patient_trials = []
                    for y in range(trial_count):
                        start_date, end_date = random_dates(diagnosed_date, cured_date)
                        patient_trial_obj = PatientTrials('tr' + str(k), str(start_date.strftime("%Y%m%d")), str(end_date.strftime("%Y%m%d")), 'pc' + str(i), 'Th' + str(random.randint(1,204)), str(random.randint(1,100)))
                        patient_trials.append(patient_trial_obj.__dict__)
                        k = k + 1
                        
                    if trial_count == 0:
                        patient_condition_obj = PatientConditions('pc' + str(j), str(diagnosed_date.strftime("%Y%m%d")), None, 'Cond' + str(random.randint(1,322)))
                    else:
                        patient_condition_obj = PatientConditions('pc' + str(j), str(diagnosed_date.strftime("%Y%m%d")), str(cured_date.strftime("%Y%m%d")), 'Cond' + str(random.randint(1,322)))
                    
                    patient_condition.append(patient_condition_obj.__dict__) 
                    j = j + 1
                    
                patient_obj = Patients(i, patient_name, patient_gender, patient_age, patient_condition, patient_trials)
                dataSet['Patients'].append(patient_obj.__dict__)
                i = i + 1

In [9]:
condition()
therapy()
patient()
with open(patientJSON, 'w') as jsonFile:
    jsonFile.write(json.dumps(dataSet, indent=4))

In [10]:
# print(pd.read_json(patientJSON), lines=True)

In [11]:
# patients= pd.read_json(patientJSON)
# patients.head()

In [12]:
# dicts = {}
# values = []

# #     for i in range(len(values)):
# #         dicts[i] = values[i]

#     return json.dumps(data['Therapies'])
#     with open(patientJSON, 'w') as jsonFile:
#         jsonFile.write(json.dumps(data, indent=4))

# print(therap())

In [13]:
# therapies= pd.read_json(therapyJSON, lines=True, orient='records')
# therapies.head()

In [14]:
# conditions= pd.read_json(conditionJSON)
# conditions.head()